### План работы

1. Выбрать два 7-10 минутных видео на английском языке (TED talk/интервью/...) с мужским и женским голосом
2. Выполнить предобработку данных: выделить аудио, сегментировать, привести к нужному  для ASR модели формату, при необходимости применить деноизинг
3. Запустить любые open-source ASR и MT модели. Объяснить выбор моделей.
4. Разобраться в архитектуре модели XTTS https://arxiv.org/pdf/2406.04904 
5. Zero-shot voice cloning: 
   - скопировать репозиторий https://github.com/coqui-ai/TTS
   - сгенерировать аудио для соответствующих текстов на русском языке аудио моделью XTTSv2. В качестве промпта подавать исходные записи на английском для копирования голоса 
   - наложить аудио на видео
6. Few-shot voice cloning:  
   - выбрать голос персонажа/актера, на котором zero-shot генерация дает не совсем точное копирование/не передает специфические характеристики, и собрать аудиозаписи для него
   - выполнить файнтюнинг модели на этих данных (проверить на разном количестве данных, подобрать гиперпараметры при необходимости)
   - сгенерировать те же предложения до и после файнтюнинга
   - посчитать объективную метрику похожести голоса (speaker smilarity): косинусное расстояние между векторами модели исходной (gt) и сгенерированной записи для zero- и few-shot вариантов

Результат представить:
  - в виде ноутбука с поэтапным описанием каждого шага/ скриптов
  - описания XTTS модели, основных компонент архитектуры и их задач. описание того, какие части обновляются во время few-shot клонирования
  - описания экспериментов по файнтюнингу, текущих проблем пайплайна
  - исходные и полученные видео

### Предобработка

In [1]:
import os
import librosa
import torch
import datetime
import json
from moviepy import VideoFileClip
from pathlib import Path
from tqdm.notebook import tqdm
from transformers import pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [7]:
def convert_video_to_audio_moviepy(video_file: str, output_ext: str = "mp3") -> None:
    """Converts video to audio using MoviePy library
    that uses `ffmpeg` under the hood"""
    filename, _ = os.path.splitext(video_file)
    clip = VideoFileClip(video_file)
    clip.audio.write_audiofile(f"{filename}.{output_ext}")

In [ ]:
for video_file in tqdm(list(Path("data").glob("*.mp4"))):
    convert_video_to_audio_moviepy(video_file)

  0%|          | 0/6 [00:00<?, ?it/s]

MoviePy - Writing audio in data/ch_broken.mp3


MoviePy - Done.
MoviePy - Writing audio in data/Tom Cruise on Performing His Own Dangerous Stunts, ‘Mission Impossible’ Training, and Cliff-Hangers (360p).mp3


MoviePy - Done.
MoviePy - Writing audio in data/Cameron's interview - CANADA - #HUMAN (360p).mp3


MoviePy - Done.
MoviePy - Writing audio in data/1987 on TODAY- Bruce Willis talks balancing fame and privacy (360p).mp3


MoviePy - Done.
MoviePy - Writing audio in data/Bruno's interview - ENGLAND - #HUMAN (360p).mp3


MoviePy - Done.
MoviePy - Writing audio in data/James May Talks- TV, Animals, Clarkson & Hammond (360p).mp3


MoviePy - Done.


### ASR

In [20]:
asr_model = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base", device=device
    # "automatic-speech-recognition", model="openai/whisper-small.en", device=device
)

Device set to use cpu


In [ ]:
def speech_to_text(model, audio_file):
    array, sampling_rate = librosa.load(audio_file)
    # trimmed_array, _ = librosa.effects.trim(array)
    intervals = librosa.effects.split(array)
    result = []
    for interval in intervals:
        result.append(
            {
                **model(
                    {
                        "array": array[interval[0]:interval[1]],
                        "sampling_rate": sampling_rate
                    },
                    chunk_length_s=30,
                ),
                "timestamp": librosa.samples_to_time(interval).tolist()
            }
        )
    return {"chunks": result}


In [ ]:
asr_model()

In [144]:
list(Path("data").glob("*.mp3"))[1].stem

'Tom Cruise on Performing His Own Dangerous Stunts, ‘Mission Impossible’ Training, and Cliff-Hangers (360p)'

In [ ]:
result = {}
for audio_file in tqdm(list(Path("data").glob("*.mp3"))):
    result[audio_file.stem] = speech_to_text(asr_model, audio_file)

  0%|          | 0/5 [00:00<?, ?it/s]

/Users/d.kulemin/Projects/dls-video-dubbing/.venv/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [189]:
with open("stt_data.json", "w") as fout:
    json.dump(result, fout, sort_keys=True, indent=4)

### MT

In [29]:
mt_model = pipeline(
    # "automatic-speech-recognition", 
    # model="Helsinki-NLP/opus-mt-en-ru",
    # model="facebook/nllb-200-distilled-600M",
    # model="facebook/wmt19-en-ru", # norm
    model="jbochi/madlad400-7b-mt",
    device=device
)

config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/101k [00:00<?, ?B/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/3.34G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/4.43M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.6M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cpu


In [4]:
stt_data = json.load(open("stt_data.json"))

In [5]:
test_text = stt_data['1987 on TODAY- Bruce Willis talks balancing fame and privacy (360p)']['chunks'][0]['text']

In [24]:
test_text.split('.')

[' Bruce Willis is one red hot property these days',
 ' As smart mouth David Addison, he stars in the much heralded TV series Moonlighting',
 " He's done a special for HBO and has a second in the works",
 " He has a record album out that's just gone gold and his first feature film Blind Date opens tomorrow",
 ' Yet for all his success in front of the camera, Willis has a less than terrific off screen image',
 ' I caught up with Bruce and Los Angeles, simply asked him why',
 " Well, I'm glad you asked that question",
 " And given the fact that in the past two and a half years since I've been doing this show, I've only done three major interviews",
 " That's Side A",
 " Side B is, for the past nine months, I've been in magazines and newspapers just about every day, or at least once a week",
 ' And there have been things that have been total fabrications printed as fact about me',
 " I'm not saying that I'm an angel",
 ' I like to have fun',
 " But by and large, I think how people see me 

In [7]:
test_text[:1000]

" Bruce Willis is one red hot property these days. As smart mouth David Addison, he stars in the much heralded TV series Moonlighting. He's done a special for HBO and has a second in the works. He has a record album out that's just gone gold and his first feature film Blind Date opens tomorrow. Yet for all his success in front of the camera, Willis has a less than terrific off screen image. I caught up with Bruce and Los Angeles, simply asked him why. Well, I'm glad you asked that question. And given the fact that in the past two and a half years since I've been doing this show, I've only done three major interviews. That's Side A. Side B is, for the past nine months, I've been in magazines and newspapers just about every day, or at least once a week. And there have been things that have been total fabrications printed as fact about me. I'm not saying that I'm an angel. I like to have fun. But by and large, I think how people see me in this country is pretty much based on what they rea

In [ ]:
stt_data

In [ ]:
mt_model("")

In [30]:
mt_model(" But by and large, I think how people see me in this country is pretty much based on what they read in the papers that I ran through town nude that I used to show up at parties with diapers on which is all real good copy but a lot of it's just been made up",)

Your input_length: 62 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


KeyboardInterrupt: 

In [ ]:
chunk["text"][:1000]

" Bruce Willis is one red hot property these days. As smart mouth David Addison, he stars in the much heralded TV series Moonlighting. He's done a special for HBO and has a second in the works. He has a record album out that's just gone gold and his first feature film Blind Date opens tomorrow. Yet for all his success in front of the camera, Willis has a less than terrific off screen image. I caught up with Bruce and Los Angeles, simply asked him why. Well, I'm glad you asked that question. And given the fact that in the past two and a half years since I've been doing this show, I've only done three major interviews. That's Side A. Side B is, for the past nine months, I've been in magazines and newspapers just about every day, or at least once a week. And there have been things that have been total fabrications printed as fact about me. I'm not saying that I'm an angel. I like to have fun. But by and large, I think how people see me in this country is pretty much based on what they rea

In [8]:
mt_model(test_text[:1000])

Your input_length: 238 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


[{'translation_text': 'Брюс Уиллис - одна из самых горячих тем в наши дни. Как умный Дэвид Аддисон, он снимается в популярном телесериале "Лунное освещение". Он сделал специально для О ". У него есть второй. У него есть пластинка, который только что есть". У него есть альбом, который только что есть, который только что есть, и его только что есть. У него есть. У него есть, и его только что есть, и его первый альбом пластинка, который только что золотой, и его первый полнометражный фильм, и его первый полнометражный фильм "Blind Date Date открывается завтра завтра". Но перед кассора ". И все перед камерой". И все ". И все", что завтра ". И все", все, что ", что". Но все ", все, что есть". И все, что есть, все, что'}]

In [ ]:
mt_model(test_text.split("."))

[{'translation_text': 'Брюс Уиллис - одна из самых горячих звезд современности'},
 {'translation_text': 'Как умный Дэвид Аддисон, он снимается в популярном телесериале "Лунное освещение"'},
 {'translation_text': 'Он сделал специально для О "О", и у него есть второй в работе'},
 {'translation_text': 'У него есть пластинка, которая только что ушла золотом, и его первый полнометражный фильм "Blind Date Date" открывается завтра ".'},
 {'translation_text': 'Тем не менее, несмотря на весь свой успех перед камерой, у Виллиса менее, изображение за кадром менее, далеко не ужасающее.'},
 {'translation_text': 'Я догнал Брюса и Лос-Анджелес, просто спросил его, почему'},
 {'translation_text': 'Что ж, я рад, что вы задали этот вопрос'},
 {'translation_text': 'А учитывая тот факт, что за последние два с половиной года, с тех пор как я делаю это шоу, я дал всего три интервью, я дал всего три крупных интервью, я дал всего три.'},
 {'translation_text': 'Это сторона А'},
 {'translation_text': 'В течение

In [ ]:
mt_model

In [10]:
for title, data in tqdm(stt_data.items()):
    for chunk in tqdm(data["chunks"]):
        # print(title, chunk["text"])
        # translated_texts = []
        # for text in tqdm(chunk["text"].split(".")):
        #     translated_texts.append(mt_model(text + "."))
        # chunk["translated"] = " ".join(translated_texts)
        chunk["translated"] = " ".join([
            output["translation_text"]
            for output in mt_model([sentence + "." for sentence in chunk["text"].split(".")])
        ])

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Your input_length: 669 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 203 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 655 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


  0%|          | 0/1 [00:00<?, ?it/s]

Your input_length: 265 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [27]:
with open("mt_data.json", "w", encoding="utf-8") as fout:
    json.dump(stt_data, fout, sort_keys=True, indent=4, ensure_ascii=False)

In [26]:
stt_data['1987 on TODAY- Bruce Willis talks balancing fame and privacy (360p)']['chunks'][0]['translated']

'Брюс Уиллис - один из самых горячих в наши дни. Как умный Дэвид Аддисон, он снимается в популярном телесериале "Лунное освещение". Он сделал специально для О "О", и у него есть второй в работе. У него есть пластинка, которая только что ушла золотом, и его первый полнометражный фильм Blind Date Date открывается завтра. Тем не менее, несмотря на весь свой успех перед камерой, у Виллиса менее, далеко не пугающее изображение за кадром. Я догнал Брюса и Лос-Анджелес, просто спросил его почему. Что ж, я рад, что вы задали этот вопрос. И учитывая тот факт, что за последние два с половиной года, с тех пор как я делаю это шоу, я, я дал всего три интервью, я дал всего три крупных интервью. Это Сила А. В течение последних девяти месяцев я появлялся в газетах и журналах почти каждый день, или хотя бы раз в неделю. И были вещи, которые были напечатаны как факты обо мне. Я не говорю, что я ангел. Мне нравится веселиться. Но по большому счету, я думаю, что то, как люди видят меня в этой стране, в зн

In [27]:
translated_texts

[[{'translation_text': 'Брюс Уиллис - один из красных горячих объектов в наши дни.'}],
 [{'translation_text': 'Как умный язык Дэвид Эддисон, он звёзды в знаменитом телесериале "Лунный свет".'}],
 [{'translation_text': 'Он сделал специальное для HBO и у него есть секунда в работе.'}],
 [{'translation_text': 'У него есть альбом, который только что стал золотым, и его первый фильм "Слепая дата" открывается завтра.'}],
 [{'translation_text': 'Тем не менее, за все свои успехи перед камерой, Уиллис имеет менее чем потрясающий снимок с экрана.'}],
 [{'translation_text': 'Я догнала Брюса и Лос-Анджелеса, просто спросила почему.'}],
 [{'translation_text': 'Я рад, что ты задал этот вопрос.'}],
 [{'translation_text': 'И учитывая тот факт, что за последние два с половиной года с тех пор, как я выступал в этом шоу, я провел только три важных интервью.'}],
 [{'translation_text': 'Это сторона А.'}],
 [{'translation_text': 'Сбоку Б: последние девять месяцев я бывал в журналах и газетах почти каждый де

In [31]:

mt_model("Machine learning is great, isn't it?")

KeyboardInterrupt: 